In [88]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sklearn.metrics
import sys
import datasets
import numpy as np
import clin.eval

sys.path.append('../experiments/')
results_dir = '../results/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
r = imodelsx.process_results.get_results_df(results_dir, use_cached=True)

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
val = pd.DataFrame.from_dict(dset['validation'])
test = pd.DataFrame.from_dict(dset['test'])
df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]

# add medication status eval
accs_cond, f1s_macro_cond = clin.eval.eval_medication_status(dfe, r)
r['acc_cond'] = accs_cond
r['f1_macro_cond'] = f1s_macro_cond

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 445.68it/s]


In [91]:
(
    r.groupby(['checkpoint', 'n_shots'])[['f1', 'recall', 'precision', 'acc_cond', 'f1_macro_cond']].mean()
    .style.format(precision=3).background_gradient(cmap='Blues')
)